***Quora Question Pair Classification Project**




**Importing important libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

**import datase**t

In [2]:
df=pd.read_csv('/content/drive/MyDrive/quora train dataset.csv')
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,Unnamed: 6,Unnamed: 7
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,NaN,NaN
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,NaN,NaN
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,NaN,NaN
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,NaN,NaN
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,NaN,NaN


Dataset Cleaning

In [3]:
df.drop(columns=['Unnamed: 6','Unnamed: 7'],inplace=True)

In [4]:
new_df=df.sample(30000,random_state=2)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
398782,398782,496695,532029,What is the best marketing automation tool for...,What is the best marketing automation tool for...,1
115086,115086,187729,187730,I am poor but I want to invest. What should I do?,I am quite poor and I want to be very rich. Wh...,0
327711,327711,454161,454162,I am from India and live abroad. I met a guy f...,T.I.E.T to Thapar University to Thapar Univers...,0
367788,367788,498109,491396,Why do so many people in the U.S. hate the sou...,My boyfriend doesnt feel guilty when he hurts ...,0
151235,151235,237843,50930,Consequences of Bhopal gas tragedy?,What was the reason behind the Bhopal gas trag...,0


**Text Preprocessing**


In [5]:
def preprocess(q):
    q=str(q).lower().strip()
    #Replace certain characters with their string equivalence
    q=q.replace('%','percent')
    q=q.replace('$','doller')
    q=q.replace('₹','rupee')
    q=q.replace('@','at')
    #the pattern "[math]" appears around 900 times in the whole dataset
    q=q.replace('[math]','')
    #replacing some numbers with string equilantents (not perfect.can be done better account for more cases)
    q=q.replace(',000,000,000','b ')
    q=q.replace(',000,000','m ')
    q=q.replace(',000','k ')
    q=re.sub(r'([0-9]+)000000000',r'\1b', q)
    q=re.sub(r'([0-9]+)000000',r'\1m', q)
    q=re.sub(r'([0-9]+)000',r'\1k', q)

    contractions = {
    "ain't": "am not / are not / is not / has not / have not",
    "aren't": "are not / am not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had / he would",
    "he'd've": "he would have",
    "he'll": "he shall / he will",
    "he'll've": "he shall have / he will have",
    "he's": "he has / he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how has / how is / how does",
    "I'd": "I had / I would",
    "I'd've": "I would have",
    "I'll": "I shall / I will",
    "I'll've": "I shall have / I will have",
    "I'm": "I am",
    "I've": "I have",
    "isn't": "is not",
    "it'd": "it had / it would",
    "it'd've": "it would have",
    "it'll": "it shall / it will",
    "it'll've": "it shall have / it will have",
    "it's": "it has / it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had / she would",
    "she'd've": "she would have",
    "she'll": "she shall / she will",
    "she'll've": "she shall have / she will have",
    "she's": "she has / she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so as / so is",
    "that'd": "that would / that had",
    "that'd've": "that would have",
    "that's": "that has / that is",
    "there'd": "there had / there would",
    "there'd've": "there would have",
    "there's": "there has / there is",
    "they'd": "they had / they would",
    "they'd've": "they would have",
    "they'll": "they shall / they will",
    "they'll've": "they shall have / they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had / we would",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what shall / what will",
    "what'll've": "what shall have / what will have",
    "what're": "what are",
    "what's": "what has / what is",
    "what've": "what have",
    "when's": "when has / when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where has / where is",
    "where've": "where have",
    "who'll": "who shall / who will",
    "who'll've": "who shall have / who will have",
    "who's": "who has / who is",
    "who've": "who have",
    "why's": "why has / why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had / you would",
    "you'd've": "you would have",
    "you'll": "you shall / you will",
    "you'll've": "you shall have / you will have",
    "you're": "you are",
    "you've": "you have"}

    q_decontracted=[]

    for word in q.split():
        if word in contractions:
            word=contractions[word]

        q_decontracted.append(word)

    q=' '.join(q_decontracted)
    q=q.replace("'ve"," have")
    q=q.replace("n't"," not")
    q=q.replace("'re"," are")
    q=q.replace("'ll"," will")

    #Removing HtMl tags
    q=BeautifulSoup(q)
    q=q.get_text()

    #remove punctuations
    pattern=re.compile('\W')
    q= re.sub(pattern,' ',q).strip()

    return q


In [6]:
preprocess("I've already! wasn't <b>done</b>?")

'i have already  was not done'

In [7]:
new_df['question1']=new_df['question1'].apply(preprocess)
new_df['question2']=new_df['question2'].apply(preprocess)

In [8]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1
115086,115086,187729,187730,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...,0
327711,327711,454161,454162,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...,0
367788,367788,498109,491396,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0
151235,151235,237843,50930,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy,0


**Feature Generation**

In [9]:
new_df['q1_len']=new_df['question1'].str.len()
new_df['q2_len']=new_df['question2'].str.len()

In [10]:
new_df['q1_num_words']=new_df['question1'].apply(lambda row:len(row.split(" ")))
new_df['q2_num_words']=new_df['question2'].apply(lambda row:len(row.split(" ")))

In [11]:
def common_words(row):
   w1=set(map(lambda word: word.lower().strip(),row['question1'].split(" ")))
   w2=set(map(lambda word: word.lower().strip(),row['question2'].split(" ")))
   return len(w1 & w2)

In [12]:
new_df['word_common']=new_df.apply(common_words,axis=1)
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,75,76,13,13,12
115086,115086,187729,187730,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...,0,48,56,13,16,8
327711,327711,454161,454162,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...,0,104,119,28,21,4
367788,367788,498109,491396,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0,58,145,14,32,1
151235,151235,237843,50930,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy,0,34,49,5,9,3


In [13]:
def total_words(row):
   w1=set(map(lambda word: word.lower().strip(),row['question1'].split(" ")))
   w2=set(map(lambda word: word.lower().strip(),row['question2'].split(" ")))
   return (len(w1) + len(w2))

In [14]:
new_df['word_total']=new_df.apply(total_words,axis=1)

In [15]:
new_df['word_share']=round(new_df['word_common']/new_df['word_total'],2)

In [16]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,75,76,13,13,12,26,0.46
115086,115086,187729,187730,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...,0,48,56,13,16,8,24,0.33
327711,327711,454161,454162,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...,0,104,119,28,21,4,38,0.11
367788,367788,498109,491396,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0,58,145,14,32,1,34,0.03
151235,151235,237843,50930,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy,0,34,49,5,9,3,13,0.23


In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
from nltk.corpus import stopwords
stop_words=stopwords.words('english')

In [19]:
#Advanced features
from nltk.corpus import stopwords

def fetch_token_features(row):

  q1=row['question1']
  q2=row['question2']

  SAFE_DIV=0.0001
  stop_words=stopwords.words('english')
  token_features=[0.0]*8
  #converting the sentence into Tokens:
  q1_tokens=q1.split()
  q2_tokens=q2.split()

  if len(q1_tokens)==0 or len(q2_tokens)==0:
      return token_features
  #get the non_stopwords in questions
  q1_words=set([word for word in q1_tokens if word not in stop_words])
  q2_words=set([word for word in q2_tokens if word not in stop_words])

  #get the stopwords in questions
  q1_stops=set([word for word in q1_tokens if word in stop_words])
  q2_stops=set([word for word in q2_tokens if word in stop_words])
  #get the common non-stopwords from question pair
  common_word_count=len(q1_words.intersection(q2_words))
  #get the common stopwords from question pair
  common_stop_count=len(q1_stops.intersection(q2_stops))
  #get the common tokens from question pair
  common_token_count=len(set(q1_tokens).intersection(set(q2_stops)))

  token_features[0]=common_word_count/(min(len(q1_words),len(q2_words))+SAFE_DIV)
  token_features[1]=common_word_count/(max(len(q1_words),len(q2_words))+SAFE_DIV)
  token_features[2]=common_stop_count/(min(len(q1_words),len(q2_words))+SAFE_DIV)
  token_features[3]=common_stop_count/(max(len(q1_words),len(q2_words))+SAFE_DIV)
  token_features[4]=common_token_count/(min(len(q1_words),len(q2_words))+SAFE_DIV)
  token_features[5]=common_token_count/(max(len(q1_words),len(q2_words))+SAFE_DIV)

  #last word of both question is same or not
  token_features[6]=int(q1_tokens[-1]==q2_tokens[-1])
  #first word of both question is same or not
  token_features[7]=int(q1_tokens[0]==q2_tokens[0])
  return token_features

In [20]:
token_features=new_df.apply(fetch_token_features,axis=1)
new_df['cwc_min']=list(map(lambda x:x[0],token_features))
new_df['cwc_max']=list(map(lambda x:x[1],token_features))
new_df['csc_min']=list(map(lambda x:x[2],token_features))
new_df['csc_max']=list(map(lambda x:x[3],token_features))
new_df['ctc_min']=list(map(lambda x:x[4],token_features))
new_df['ctc_max']=list(map(lambda x:x[4],token_features))
new_df['last_word_eq']=list(map(lambda x:x[5],token_features))
new_df['first_word_eq']=list(map(lambda x:x[6],token_features))

In [21]:
new_df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,...,word_total,word_share,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,75,76,13,13,...,26,0.46,0.874989,0.874989,0.624992,0.624992,0.624992,0.624992,0.624992,1.0
115086,115086,187729,187730,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...,0,48,56,13,16,...,24,0.33,0.666644,0.499988,1.666611,1.249969,1.666611,1.666611,1.249969,1.0
327711,327711,454161,454162,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...,0,104,119,28,21,...,38,0.11,0.000000,0.000000,0.333330,0.333330,0.333330,0.333330,0.333330,0.0
367788,367788,498109,491396,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0,58,145,14,32,...,34,0.03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
151235,151235,237843,50930,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy,0,34,49,5,9,...,13,0.23,0.749981,0.599988,0.000000,0.000000,0.000000,0.000000,0.000000,1.0


In [22]:
!pip install distance

In [23]:
import distance

def fetch_length_features(row):

    q1=row['question1']
    q2=row['question2']

    length_features = [0.0]*3

    #converting the sentence into tokens:
    q1_tokens=q1.split()
    q2_tokens=q2.split()

    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return length_features

    #absoulute Length features
    length_features[0] = abs(len(q1_tokens)-len(q2_tokens))
    #averages Token length of both questions
    length_features[1] = (len(q1_tokens) + len(q2_tokens))/2

    strs=list(distance.lcsubstrings(q1, q2))
    length_features[2] = len(strs[0]) / (min(len(q1), len(q2)) +1)

    return length_features

In [24]:
length_features = new_df.apply(fetch_length_features,axis=1)

new_df['abs_len_diff']=list(map(lambda x:x[0],length_features))
new_df['mean_len']=list(map(lambda x:x[1],length_features))
new_df['longest_substr_ratio']=list(map(lambda x:x[2],length_features))


In [25]:
!pip install fuzzywuzzy

In [26]:
#Fuzzy Features
from fuzzywuzzy import fuzz

def fetch_fuzzy_features(row):

  q1=row['question1']
  q2=row['question2']

  fuzzy_features=[0.0]*4

  #fuzz ratio
  fuzzy_features[0]=fuzz.QRatio(q1,q2)

  #fuzz_partial_ratio
  fuzzy_features[1]=fuzz.QRatio(q1,q2)

  #token_sort_ratio
  fuzzy_features[2]=fuzz.token_sort_ratio(q1,q2)

  #token_set_ratio
  fuzzy_features[3]=fuzz.token_set_ratio(q1,q2)

  return fuzzy_features



In [27]:
fuzzy_features = new_df.apply(fetch_fuzzy_features, axis=1)

new_df['fuzz_ratio']=list(map(lambda x:x[0],fuzzy_features))
new_df['fuzz_partial_ratio']=list(map(lambda x:x[1],fuzzy_features))
new_df['token_sort_ratio']=list(map(lambda x:x[2],fuzzy_features))
new_df['token_set_ratio']=list(map(lambda x:x[3],fuzzy_features))


In [28]:
def test_common_words(q1,q2):
    w1 = set(map(lambda word: word.lower().strip(), q1.split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), q2.split(" ")))
    return len(w1 & w2)

In [29]:
def test_total_words(q1,q2):
    w1 = set(map(lambda word: word.lower().strip(), q1.split(" ")))
    w2 = set(map(lambda word: word.lower().strip(), q2.split(" ")))
    return (len(w1) + len(w2))

In [30]:
def test_fetch_token_features(q1,q2):

    SAFE_DIV = 0.0001

    STOP_WORDS = stopwords.words("english")

    token_features = [0.0]*8

    # Converting the Sentence into Tokens:
    q1_tokens = q1.split()
    q2_tokens = q2.split()

    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return token_features

    # Get the non-stopwords in Questions
    q1_words = set([word for word in q1_tokens if word not in STOP_WORDS])
    q2_words = set([word for word in q2_tokens if word not in STOP_WORDS])

    #Get the stopwords in Questions
    q1_stops = set([word for word in q1_tokens if word in STOP_WORDS])
    q2_stops = set([word for word in q2_tokens if word in STOP_WORDS])

    # Get the common non-stopwords from Question pair
    common_word_count = len(q1_words.intersection(q2_words))

    # Get the common stopwords from Question pair
    common_stop_count = len(q1_stops.intersection(q2_stops))

    # Get the common Tokens from Question pair
    common_token_count = len(set(q1_tokens).intersection(set(q2_tokens)))


    token_features[0] = common_word_count / (min(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[1] = common_word_count / (max(len(q1_words), len(q2_words)) + SAFE_DIV)
    token_features[2] = common_stop_count / (min(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[3] = common_stop_count / (max(len(q1_stops), len(q2_stops)) + SAFE_DIV)
    token_features[4] = common_token_count / (min(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)
    token_features[5] = common_token_count / (max(len(q1_tokens), len(q2_tokens)) + SAFE_DIV)

    # Last word of both question is same or not
    token_features[6] = int(q1_tokens[-1] == q2_tokens[-1])

    # First word of both question is same or not
    token_features[7] = int(q1_tokens[0] == q2_tokens[0])

    return token_features

In [31]:
def test_fetch_length_features(q1,q2):

    length_features = [0.0]*3

    # Converting the Sentence into Tokens:
    q1_tokens = q1.split()
    q2_tokens = q2.split()

    if len(q1_tokens) == 0 or len(q2_tokens) == 0:
        return length_features

    # Absolute length features
    length_features[0] = abs(len(q1_tokens) - len(q2_tokens))

    #Average Token Length of both Questions
    length_features[1] = (len(q1_tokens) + len(q2_tokens))/2

    strs = list(distance.lcsubstrings(q1, q2))
    length_features[2] = len(strs[0]) / (min(len(q1), len(q2)) + 1)

    return length_features

In [32]:
def test_fetch_fuzzy_features(q1,q2):

    fuzzy_features = [0.0]*4

    # fuzz_ratio
    fuzzy_features[0] = fuzz.QRatio(q1, q2)

    # fuzz_partial_ratio
    fuzzy_features[1] = fuzz.partial_ratio(q1, q2)

    # token_sort_ratio
    fuzzy_features[2] = fuzz.token_sort_ratio(q1, q2)

    # token_set_ratio
    fuzzy_features[3] = fuzz.token_set_ratio(q1, q2)

    return fuzzy_features

In [33]:
def query_point_creator(q1,q2):
     input_query=[]
  #preprocess
     q1=preprocess(q1)
     q2=preprocess(q2)

     #fetch basic features
     input_query.append(len(q1))
     input_query.append(len(q2))

     input_query.append(len(q1.split(" ")))
     input_query.append(len(q2.split(" ")))

     input_query.append(test_common_words(q1,q2))
     input_query.append(test_total_words(q1,q2))
     input_query.append(round(test_common_words(q1,q2)/test_common_words(q1,q2),2))

     #fetch token features
     token_features=test_fetch_token_features(q1,q2)
     input_query.extend(token_features)

     #fetch length based features
     length_features=test_fetch_length_features(q1,q2)
     input_query.extend(length_features)

     #fetch fuzzy features
     fuzzy_features=test_fetch_fuzzy_features(q1,q2)
     input_query.extend(fuzzy_features)

     #bow feature for q1
     q1_bow=cv.transform([q1]).toarray()

     #bow feature for q2
     q2_bow=cv.transform([q2]).toarray()

     return np.hstack((np.array(input_query).reshape(1,22),q1_bow,q2_bow))








In [34]:
print(new_df.shape)
new_df.head()

(30000, 28)


,id,qid1,qid2,question1,question2,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
398782,398782,496695,532029,what is the best marketing automation tool for...,what is the best marketing automation tool for...,1,75,76,13,13,...,0.624992,0.624992,1.0,0.0,13.0,0.855263,99,99,99,99
115086,115086,187729,187730,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...,0,48,56,13,16,...,1.666611,1.249969,1.0,3.0,13.5,0.224490,69,69,65,74
327711,327711,454161,454162,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...,0,104,119,28,21,...,0.333330,0.333330,0.0,6.0,23.0,0.047619,26,26,34,43
367788,367788,498109,491396,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...,0,58,145,14,32,...,0.000000,0.000000,0.0,17.0,21.5,0.050847,29,29,23,30
151235,151235,237843,50930,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy,0,34,49,5,9,...,0.000000,0.000000,1.0,4.0,7.0,0.542857,55,55,48,69


In [35]:
#sns.pairplot(new_df[['ctc_min','cwc_min','csc_min','is_duplicate']],hue='is_duplicate')

In [36]:
que_df=new_df[['question1','question2']]
que_df.head()

,question1,question2
398782,what is the best marketing automation tool for...,what is the best marketing automation tool for...
115086,i am poor but i want to invest what should i do,i am quite poor and i want to be very rich wh...
327711,i am from india and live abroad i met a guy f...,t i e t to thapar university to thapar univers...
367788,why do so many people in the u s hate the sou...,my boyfriend doesnt feel guilty when he hurts ...
151235,consequences of bhopal gas tragedy,what was the reason behind the bhopal gas tragedy


In [37]:
final_df=new_df.drop(columns=['id','qid1','qid2','question1','question2'])
print(final_df.shape)
final_df.head()

(30000, 23)


,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,cwc_min,cwc_max,...,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,longest_substr_ratio,fuzz_ratio,fuzz_partial_ratio,token_sort_ratio,token_set_ratio
398782,1,75,76,13,13,12,26,0.46,0.874989,0.874989,...,0.624992,0.624992,1.0,0.0,13.0,0.855263,99,99,99,99
115086,0,48,56,13,16,8,24,0.33,0.666644,0.499988,...,1.666611,1.249969,1.0,3.0,13.5,0.224490,69,69,65,74
327711,0,104,119,28,21,4,38,0.11,0.000000,0.000000,...,0.333330,0.333330,0.0,6.0,23.0,0.047619,26,26,34,43
367788,0,58,145,14,32,1,34,0.03,0.000000,0.000000,...,0.000000,0.000000,0.0,17.0,21.5,0.050847,29,29,23,30
151235,0,34,49,5,9,3,13,0.23,0.749981,0.599988,...,0.000000,0.000000,1.0,4.0,7.0,0.542857,55,55,48,69


In [38]:
from sklearn.feature_extraction.text import CountVectorizer
#merge texts
questions=list(que_df['question1'])+list(que_df['question2'])
cv=CountVectorizer(max_features=3000)
q1_arr,q2_arr=np.vsplit(cv.fit_transform(questions).toarray(),2)

In [39]:
temp_df1=pd.DataFrame(q1_arr,index=que_df.index)
temp_df2=pd.DataFrame(q2_arr,index=que_df.index)
temp_df=pd.concat([temp_df1,temp_df2],axis=1)

In [40]:
final_df=pd.concat([final_df,temp_df],axis=1)
print(final_df.shape)
final_df.head()

(30000, 6023)


,is_duplicate,q1_len,q2_len,q1_num_words,q2_num_words,word_common,word_total,word_share,cwc_min,cwc_max,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
398782,1,75,76,13,13,12,26,0.46,0.874989,0.874989,...,0,0,0,0,0,0,0,0,0,0
115086,0,48,56,13,16,8,24,0.33,0.666644,0.499988,...,0,0,0,0,0,0,0,0,0,0
327711,0,104,119,28,21,4,38,0.11,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
367788,0,58,145,14,32,1,34,0.03,0.000000,0.000000,...,0,0,0,1,0,0,0,0,0,0
151235,0,34,49,5,9,3,13,0.23,0.749981,0.599988,...,0,0,0,0,0,0,0,0,0,0


**Model Modeling**

In [75]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(final_df.iloc[:,1:].values,final_df.iloc[:,0],test_size=0.3,random_state=1)

In [81]:
from sklearn.metrics import accuracy_score,classification_report
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.model_selection import GridSearchCV

**logistic Regression algorithm**

In [76]:
classifier=LogisticRegression()
from sklearn.model_selection import GridSearchCV
parameter={'penalty':['l1','l2','elasticnet'],'max_iter':[10,15,20]}

In [77]:
classifier_regressor=GridSearchCV(classifier,param_grid=parameter,scoring='accuracy',cv=3)

In [78]:
pipe=Pipeline([
    ('classifier_regressor',classifier_regressor)
])

In [79]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('classifier_regressor',
                 GridSearchCV(cv=3, estimator=LogisticRegression(),
                              param_grid={'max_iter': [10, 15, 20],
                                          'penalty': ['l1', 'l2',
                                                      'elasticnet']},
                              scoring='accuracy'))])

In [80]:
y_pred=pipe.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.6617777777777778


In [82]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.71      0.78      0.74      5690
           1       0.55      0.46      0.50      3310

    accuracy                           0.66      9000
   macro avg       0.63      0.62      0.62      9000
weighted avg       0.65      0.66      0.65      9000



**Decision Tree Algorithm**

In [84]:
parameter={'criterion':['ginni','entropy','log_loss'],'splitter':['best','random'],'max_depth':[4,5,6],'max_features':['auto','sqrt','log2']}
trf3=DecisionTreeClassifier()
cv=GridSearchCV(trf3,param_grid=parameter,scoring='accuracy',cv=3)

In [85]:
pipe=Pipeline([
    ('cv',cv)
])

In [86]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('cv',
                 GridSearchCV(cv=3, estimator=DecisionTreeClassifier(),
                              param_grid={'criterion': ['ginni', 'entropy',
                                                        'log_loss'],
                                          'max_depth': [4, 5, 6],
                                          'max_features': ['auto', 'sqrt',
                                                           'log2'],
                                          'splitter': ['best', 'random']},
                              scoring='accuracy'))])

In [87]:
y_pred=pipe.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.6505555555555556


In [88]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.66      0.91      0.77      5690
           1       0.57      0.20      0.29      3310

    accuracy                           0.65      9000
   macro avg       0.62      0.56      0.53      9000
weighted avg       0.63      0.65      0.59      9000



**RandomForest Algorithm**

In [91]:
trf5=RandomForestClassifier()

In [92]:
rf_params={'max_depth':[2,3,None,4],'max_features':[2,3,'auto'],'min_samples_split':[1,2,3,4],'n_estimators':[2,3,4]}

In [93]:
cv=GridSearchCV(trf5,param_grid=rf_params,scoring='accuracy',cv=3)

In [94]:
pipe=Pipeline([
    ('cv',cv)
])

In [66]:
#pipe=make_pipeline(cv)

In [95]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('cv',
                 GridSearchCV(cv=3, estimator=RandomForestClassifier(),
                              param_grid={'max_depth': [2, 3, None, 4],
                                          'max_features': [2, 3, 'auto'],
                                          'min_samples_split': [1, 2, 3, 4],
                                          'n_estimators': [2, 3, 4]},
                              scoring='accuracy'))])

In [96]:
y_pred=pipe.predict(X_test)

In [97]:
print(accuracy_score(y_test,y_pred))

0.7388888888888889


In [98]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.86      0.81      5690
           1       0.69      0.53      0.60      3310

    accuracy                           0.74      9000
   macro avg       0.72      0.70      0.70      9000
weighted avg       0.73      0.74      0.73      9000



**K-Nearest Neighbors Algorithm**

In [99]:
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train,y_train)
y_pred=neigh.predict(X_test)
accuracy_score(y_test,y_pred)

0.6831111111111111

In [100]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.74      0.76      0.75      5690
           1       0.57      0.55      0.56      3310

    accuracy                           0.68      9000
   macro avg       0.66      0.65      0.66      9000
weighted avg       0.68      0.68      0.68      9000



In [89]:
import pickle
pickle.dump(pipe,open('model.pkl','wb'))
pickle.dump(cv,open('cv.pkl','wb'))

In [90]:
pickle.dump(stop_words,open('stopwords.pkl','wb'))

**So here from observation ,i got to know that the Randomforest algorithm works very well for this project .Because on the basis of accuracy so the randomforest algorithm has highest accuracy i.e 73 % .And from classification report we can say that randomforest is the best fit model. **

In [ ]:
#################################END#############################################